# **Comentarios**

## Análisis Exploratorio de Datos
Deben visualizar la relación entre las diferentes características y la variable objetivo ("Age"). Además, deben verificar la correlación entre las características y la variable objetivo.

**Comentarios**

Aunque se eliminan los datos con la categoría "Indeterminado (I)" en el sexo del cangrejo, no se toma en cuenta que esta categoría podría tener la correlación más alta con la edad. Este detalle podría ser crucial para la selección de características y podría proporcionar información más precisa para el modelo.

A pesar de identificar que el peso del caparazón tiene la correlación más alta con la edad, no se profundiza en este punto ni se explora cómo podría afectar el rendimiento del modelo a futuro.

Realizan los graficos de dispersión para visualizar las relaciones entre las características y la edad, pero no hay un análisis detallado o discusión de estos gráficos


**Puntos o totales: 6/10**

## Construcción del Modelo:
Deben construir un modelo de regresión, que puede ser lineal múltiple o polinomial, para predecir la edad del cangrejo. Deben evaluar el modelo utilizando métricas como RMSE y MAE.


**Comentarios**

#### **Selección de Características**

Se implementa un modelo de regresión lineal sin realizar un análisis exhaustivo de los datos previamente. Aunque emplean la validación cruzada para evaluar el modelo, no consideran aspectos cruciales como la varianza en los resultados del MAE.
Si bien el promedio del MAE podría ofrecer una visión general del rendimiento del modelo, ignorar la varianza asociada deja una evaluación incompleta del modelo. Este enfoque simplificado puede llevar a conclusiones erróneas sobre la eficacia del modelo

#### **Validación**

 El razonamiento entre entre Lasso y regresión lineal no está claramente articulado. Aunque señalan que el MAE es más "uniforme" en uno de los modelos, esta afirmación es ambigua y no se respalda con evidencia concreta o análisis detallado. En realidad, la uniformidad del MSE por sí sola no es un indicador suficiente para determinar la superioridad de un modelo sobre otro.

La afirmación de que "La varianza de nuestra variable objetivo es decentemente explicada por nuestro modelo" carece de soporte empírico, ya que no se muestra ningún cálculo o métrica de varianza.

En cuanto a la elección del modelo Lasso como el mejor, la justificación para esta decisión no está claramente establecida. Generalmente, la elección de un modelo debería basarse en una comparación exhde las metricas.

**Puntos o totales: 17/25**


##Entrenamiento Final y Evaluación:
en el entrenamiento los coeficientes del modelo Lasso cambian según lo planeado. Si los coeficientes no son replicables en diferentes ejecuciones o subconjuntos del mismo conjunto de datos, esto podría indicar una falta de robustez en el modelo.

**Puntos o totales: 8/15**




In [8]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/commaseparatedvalues/CrabAgePrediction_Subset1.csv"
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, LeavePOut
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.utils import resample

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Punto 1

### 1.A Analisis exploratorio de los datos

In [9]:
df = pd.read_csv(path)
df

FileNotFoundError: ignored

In [ ]:
#como solo se pide que hagamos prediccion de machos y hembras, vamos a quitar
#los indeterminados
df = df[df['Sex'] != "I"]
df

In [ ]:
#Quitemos la variable objetivo y la variable categorica de nuestros predictores
X = df.drop(['Age', 'Sex'], axis=1)
features = X.columns.tolist()
target = df.Age
targetString = "target"

In [ ]:
num_plots = len(features)
num_cols = 2
num_rows = (num_plots + 1) // num_cols


plt.figure(figsize=(15, 5 * num_rows))


for i, feature in enumerate(features):
    plt.subplot(num_rows, num_cols, i + 1)
    sns.scatterplot(data=df, x=feature, y=target)
    plt.title(feature+" vs age")
    plt.xlabel(feature)
    plt.ylabel("target")


plt.tight_layout()
plt.show()

Hagamos la matriz de correlación, es verdad que se nos pide que solo encontremos la correlación de la variable objetivo con el resto, pero sería interesante ver si hay variables altamente correlacionadas para tenerlo en cuenta en el modelo

In [ ]:
#sacamos la variable categórica para graficar las correlaciones
test = df.drop(['Sex'], axis=1)
corr = test.corr ()
corr.style.background_gradient (cmap = 'coolwarm')

Podemos ver a lo largo de la octava columna las correlaciones de nuestra variable objetivo con el resto, la mayor correlación con la edad es la del peso del caparazón, por lo que entre más alta sea el peso del caparazón mas viejo será el cangrejo, para el resto de variables también tenemos una correlación positiva, también podemos ver que hay una correlación casi que lineal entre Length, Diameter, Weight, Shucked Weight, Viscera Weight, Shell, Weight dos a dos, evaluaremos si es pertinente usar todas las variables como predictores o si podemos generar un modelo con el diámetro y la altura para predecir la edad

### 1.B. Construye un modelo de regresión (lineal múltiple o polinomial) para predecir la edad del cangrejo y evalúa los modelos utilizando métricas como RMSE y MAE

Primero tratemos de entrenar un modelo de regresión lineal simple, es probable que no obtengamos una buena predicción con solo este modelo, en dado caso trataremos con otros, para esto usaremos la libreria scikit learn y su modelo de regresión lineal

Partamos el dataset en dos, una porción para entrenar el modelo y otro para testearlo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,target , random_state = 6,test_size = 0.3, train_size = 0.7, shuffle=True)

In [ ]:
regr = LinearRegression()
modelo = regr.fit(X_train, y_train)

Veamos que coeficientes de le asigno a cada uno de nuestros predictores al entrenar el modelo

In [ ]:
X_lista = X.columns.tolist()
coef_lista=regr.coef_.tolist()
coeficientes = {}
for i in range(len(X.columns)):
  coeficientes[X_lista[i]] = coef_lista[i]
print("coeficientes:", coeficientes)

# coeficientes: {'Length': -2.509212214254984, 'Diameter': 5.1520174188254755, 'Height': 1.726571598049385, 'Weight': 0.2834294804186035, 'Shucked Weight': -0.6884829579029614, 'Viscera Weight': -0.25324745142163896, 'Shell Weight': 0.4394334473670418}


Para Nuestros modelos usaremos el MAE:

*   Queremos saber que tanto se desvían nuestros datos predecidos respecto a los datos originales, en promedio cual es el error de predicción en años de nuestro modelo









In [ ]:
y_predict = modelo.predict(X_test)

La varianza de nuestra variable objetivo es decentemente explicada por nuestro modelo

In [ ]:
print("Obtenemos un MAE de  ", mean_absolute_error(y_test, y_predict))

# Obtenemos un MAE de   1.774496810419058

En promedio el modelo se está equivocando en 1.77 años.

Hagamos cross validation para evaluar la consistencia de nuestro modelo

In [ ]:
custom_scoring = make_scorer(mean_absolute_error)  #vamos a hacer validacion cruzada con MAE para evaluar el modelo
cv_scores = cross_val_score(regr, X_train, y_train, cv=10, scoring=custom_scoring)
plt.hist(cv_scores, bins=10, edgecolor='k')


plt.xlabel('Cross-Validation Scores')
plt.ylabel('Frequency')
plt.title('Distribution of Cross-Validation Scores')

# Mostrar el histograma.
plt.show()

In [ ]:
cv_scores

Hagamos una regresión de Lasso, queremos simplificar nuestro modelo, y dado que todo esta muy correlacionado, es posible que podamos quitar predictores que esten correlacionados entre si, usaremos validación cruzada para encontrar el mejor hiperparámetro

In [ ]:
lambdas = 10**np.linspace(10,-2,100)*0.5

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
best_alpha = None
best_mae = float('inf')
for alpha in lambdas:
    model = Lasso(alpha=alpha)
    mae_scores = cross_val_score(model, X_test, y_test, cv=kf, scoring=custom_scoring)
    mean_mae = np.mean(mae_scores)
    if mean_mae < best_mae:
        best_alpha = alpha
        best_mae = mean_mae

In [ ]:
lasso = Lasso(alpha = best_alpha)
lasso.fit(X_train, y_train)
y_predict = lasso.predict(X_test)

print("Obtenemos un MAE de  ", mean_absolute_error(y_test, y_predict))

Podemos ver que las variables más relevantes para el modelo son Diameter, y height, donde height era una de las variables que estaba menos correlacionada con el resto

In [ ]:
coef_lista=lasso.coef_.tolist()
coeficientes = {}
for i in range(len(X.columns)):
  coeficientes[X_lista[i]] = coef_lista[i]
print("coeficientes:", coeficientes)

Hagamos cross validation y ploteemos la frecuencia del MAE

In [ ]:
cv_scores = cross_val_score(lasso, X_train, y_train, cv=10, scoring=custom_scoring)
plt.hist(cv_scores, bins=10, edgecolor='k')


plt.xlabel('Cross-Validation Scores')
plt.ylabel('Frequency')
plt.title('Distribution of Cross-Validation Scores')

# Display the histogram
plt.show()

Tiene una distribución más uniforme que la regresión lineal, por lo que es un modelo más consistente.

### 1.C. Validación y entrenamiento final

Vamos a usar una regresión de Lasso, a pesar de que haya obtenido un MAE ligeramente peor que la regresión lineal, durante nuestro training y testing, es un modelo más simple, y además más consistente

In [ ]:
lasso2 = Lasso(alpha = best_alpha)
lasso2.fit(X, target)

A continuación se muestran los coeficientes de nuestro modelo

In [ ]:
coef_lista=lasso2.coef_.tolist()
X_lista = X.columns.tolist()
coeficientes = {}
for i in range(len(X.columns)):
  coeficientes[X_lista[i]] = coef_lista[i]
print("coeficientes:", coeficientes)

#coeficientes: {'Length': 0.0, 'Diameter': 1.4657491948723016, 'Height': 0.16013289686616677, 'Weight': 0.22193120478485973, 'Shucked Weight': -0.6069788619722464, 'Viscera Weight': -0.1544572199700522, 'Shell Weight': 0.5227125566912905}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
import pandas as pd
crab_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023-2S-AAD-1Corte/datos_validar/CrabAgePrediction_Subset1.csv').dropna()
crab_data_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023-2S-AAD-1Corte/datos_validar/CrabAgePrediction_Subset2.csv').dropna()
X=crab_data.drop(columns=['Age','Sex'])
target = crab_data.Age

best_alpha=0.01155064850041579

X_train_val = crab_data.drop(columns=['Age','Sex'])
y_train_val = crab_data['Age']
X_test = crab_data_test.drop(columns=['Age','Sex'])
y_test = crab_data_test['Age']

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error

lasso2=Lasso(alpha=0.01155064850041579)
lasso2.fit(X_train_val, y_train_val)

y_test_pred = lasso2.predict(X_test)


# Calcular RMSE y MAE
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'RMSE en el conjunto de prueba: {rmse_test}')
print(f'MAE en el conjunto de prueba: {mae_test}')

lasso2.coef_




RMSE en el conjunto de prueba: 2.259158733762671
MAE en el conjunto de prueba: 1.6352235017335246


array([ 0.        ,  3.95585412,  1.0222679 ,  0.17598821, -0.56571671,
       -0.12718934,  0.5559778 ])

In [ ]:
df = crab_data[crab_data['Sex'] != "I"]
